In [13]:
import sys
import os
import argparse
from multiprocessing import Process, Queue
from typing import List, Dict
import subprocess
import mmcv
import time
import csv
import matplotlib.pyplot as plt

In [30]:
parser = argparse.ArgumentParser()
parser.add_argument("--gpus", "-g", type=str, default="0",
                            help="space delimited GPU id list (global id in nvidia-smi, "
                                 "not considering CUDA_VISIBLE_DEVICES)")
parser.add_argument('--eval', action='store_true', default=False,
                   help='evaluation mode (run the render_imgs script)')
parser.add_argument('--configname',  default='syn_4096code',
                   help='a.k.a config  in subfloer ./configs/batchtest/<configname>.txt ')

parser.add_argument("--importance_include",  type=float,  default=0.6,
        help='fully vector quantize the full model')

parser.add_argument("--importance_prune",  type=float,  default=0.999,
        help='fully vector quantize the full model')

parser.add_argument("--codebook_size",  type=int,  default=4096,
        help='fully vector quantize the full model')

parser.add_argument("--dump_images",  action='store_true', default=False,
        help='fully vector quantize the full model')

parser.add_argument('--dataset', type=str, default='syn', choices=['syn', 'llff', 'tnt', 'nsvf'])
parser.add_argument('-f', type=str)
args = parser.parse_args()

PSNR_FILE_NAME = 'test_psnr.txt'

DatasetSetting={
    "syn": {
        "data": "./data/nerf_synthetic",
        "cfg": f"./configs/batch_test/{args.configname}.py",
        "basedir":f"./logs/{args.configname}",
        "scene_list": ['chair'] #['chair', 'drums', 'ficus', 'hotdog', 'lego', 'materials', 'mic', 'ship']
    },
    "tnt":{
        "data": "./data/TanksAndTemple",
        "cfg": f"./configs/batch_test/{args.configname}.py",
        "basedir":f"./logs/{args.configname}",
        "scene_list":['Barn', 'Caterpillar', 'Family', 'Ignatius', 'Truck']
    },
    "nsvf":{
        "data": "./data/Synthetic_NSVF",
        "cfg": f"./configs/batch_test/{args.configname}.py",
        "basedir":f"./logs/{args.configname}",
        "scene_list":['Bike', 'Lifestyle', 'Palace', 'Robot', 'Spaceship', 'Steamtrain', 'Toad', 'Wineholder',]
    }
    
}

datasetting = DatasetSetting[args.dataset]

class AverageMeter(object):
    def __init__(self, name=''):
        self.name=name
        self.reset()
    def reset(self):
        self.val=0
        self.sum=0
        self.avg=0
        self.count=0
    def update(self,val,n=1):
        self.val=val
        self.sum += val*n
        self.count += n
        self.avg=self.sum/self.count
    def __repr__(self) -> str:
        return f'{self.name}: average {self.count}: {self.avg}\n'

from prettytable import PrettyTable




In [36]:

for depth in range(2,9):
    width = 128
    
    table = PrettyTable(["Scene", "PSNR", "SSIM", "LPIPS_ALEX","LPIPS_VGG", "SIZE", "TIME"])
    table.float_format = '.3'
    
    PSNR=AverageMeter('PSNR')
    SSIM=AverageMeter('SSIM')
    LPIPS_A=AverageMeter('LPIPS_A')
    LPIPS_V=AverageMeter('LPIPS_V')
    SIZE=AverageMeter('SIZE')

    for scene in datasetting["scene_list"]: #[ 'chair', 'drums', 'ficus', 'hotdog', 'lego', 'mic', 'materials', 'ship'   ]:
        
        path = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/time.csv'
        with open(path, 'r') as f:
            times = [r for r in csv.DictReader(f)][0]

        path = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/render_test_vq_last/mean.txt'
        with open(path, 'r') as f:
            lines = f.readlines()
            psnr = float(lines[0].strip())
            ssim = float(lines[1].strip())
            lpips_a = float(lines[2].strip())
            lpips_v = float(lines[3].strip())
            PSNR.update(psnr)
            SSIM.update(ssim)
            LPIPS_A.update(lpips_a)
            LPIPS_V.update(lpips_v)
            compressed_file = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/extreme_saving.zip'
            if os.path.exists(compressed_file):
                size = os.path.getsize(compressed_file)/(1024*1024)
            else:
                size = 0
            table.add_row([scene, psnr, ssim, lpips_a, lpips_v, size, times['eps_time']])
            SIZE.update(size)
    table.add_row(['Mean', PSNR.avg, SSIM.avg, LPIPS_A.avg,LPIPS_V.avg, SIZE.avg, "--"])

    txt_file = os.path.join(datasetting["basedir"] + f"_d{depth}_w{width}", f'merge_{args.importance_include}_{args.importance_prune}_{args.codebook_size}.txt')
    with open(txt_file, 'w') as f:
        f.writelines(table.get_string())

    csv_file = os.path.join(datasetting["basedir"] + f"_d{depth}_w{width}", f'merge_{args.importance_include}_{args.importance_prune}_{args.codebook_size}.csv')
    with open(csv_file, 'w', newline='') as f:
        f.writelines(table.get_csv_string())
    print('VQ-DVGO:')
    print(table)


    table = PrettyTable(["Scene", "PSNR", "SSIM", "LPIPS_ALEX","LPIPS_VGG", "SIZE", "TIME"])
    table.float_format = '.3'

    PSNR=AverageMeter('PSNR')
    SSIM=AverageMeter('SSIM')
    LPIPS_A=AverageMeter('LPIPS_A')
    LPIPS_V=AverageMeter('LPIPS_V')
    SIZE=AverageMeter('SIZE')


    for scene in datasetting["scene_list"]: #[ 'chair', 'drums', 'ficus', 'hotdog', 'lego', 'mic', 'materials', 'ship'   ]:
        path = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/time.csv'
        with open(path, 'r') as f:
            times = [r for r in csv.DictReader(f)][0]
            
        path = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/render_test_fine_last/mean.txt'

        with open(path, 'r') as f:
            lines = f.readlines()
            psnr = float(lines[0].strip())
            ssim = float(lines[1].strip())
            lpips_a = float(lines[2].strip())
            lpips_v = float(lines[3].strip())
            PSNR.update(psnr)
            SSIM.update(ssim)
            LPIPS_A.update(lpips_a)
            LPIPS_V.update(lpips_v)
            compressed_file = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/fine_model.pth.zip'
            if os.path.exists(compressed_file):
                size = os.path.getsize(compressed_file)/(1024*1024)
            else:
                size = 0
            table.add_row([scene, psnr, ssim, lpips_a, lpips_v, size, times['eps_fine']])
            SIZE.update(size)
    table.add_row(['Mean', PSNR.avg, SSIM.avg, LPIPS_A.avg,LPIPS_V.avg, SIZE.avg, 0])

    txt_file = os.path.join(datasetting["basedir"] + f"_d{depth}_w{width}", f'merge_fine.txt')
    with open(txt_file, 'w') as f:
        f.writelines(table.get_string())

    csv_file = os.path.join(datasetting["basedir"] + f"_d{depth}_w{width}", f'merge_fine.csv')
    with open(csv_file, 'w', newline='') as f:
        f.writelines(table.get_csv_string())
    print('orginal DVGO:')
    print(table)

VQ-DVGO:
+-------+--------+-------+------------+-----------+-------+----------+
| Scene |  PSNR  |  SSIM | LPIPS_ALEX | LPIPS_VGG |  SIZE |   TIME   |
+-------+--------+-------+------------+-----------+-------+----------+
| chair | 33.294 | 0.970 |   0.037    |   0.022   | 1.086 | 00:05:24 |
|  Mean | 33.294 | 0.970 |   0.037    |   0.022   | 1.086 |    --    |
+-------+--------+-------+------------+-----------+-------+----------+
orginal DVGO:
+-------+--------+-------+------------+-----------+---------+----------+
| Scene |  PSNR  |  SSIM | LPIPS_ALEX | LPIPS_VGG |   SIZE  |   TIME   |
+-------+--------+-------+------------+-----------+---------+----------+
| chair | 33.577 | 0.973 |   0.031    |   0.020   | 106.362 | 00:01:58 |
|  Mean | 33.577 | 0.973 |   0.031    |   0.020   | 106.362 |    0     |
+-------+--------+-------+------------+-----------+---------+----------+


FileNotFoundError: [Errno 2] No such file or directory: './logs/syn_4096code_d3_w128/syn_4096code_chair/render_test_vq_last/mean.txt'

In [37]:
import torch
print(torch.backends.cudnn.version())

8302
